In [ ]:
print('hello world'),

In [ ]:
import numpy as np
import xsimlab as xs 
import matplotlib.pyplot as plt

In [ ]:
plt.plot(np.arange(123))

## Structure
1. basic ODE calc, present the early phytoplankton model
2. move to NPZD model, show dynamics of basic functions, plot interactively
3. then show model dynamics with a flat seasonal cycle, then add articifial cycles, then show WOA2018 forcing from some location
4. done!

In [ ]:
mu = 1
gr = 0.1

def model_one(y,t):
    P = y
    P = (mu - gr) * P
    return P

In [ ]:
@xs.process
class 